# Setup and run Bayesian inference on a batch of GW injections using `pycbc_inference`

## Setup

In [2]:
import os

run_dir = '/home/prayush/research/test_pycbc_inj'
try:
    os.makedirs(run_dir)
except:
    pass
os.chdir(run_dir)

In [3]:
!pwd

/home/prayush/research/test_pycbc_inj


In [4]:
!rm -rf *

In [5]:
!ls

## Generate sampler / inference config files

In [6]:
!gwnr_write_pycbc_inference_configs -h

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_write_pycbc_inference_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a set of signals with Pycbc inference

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-data-config WRITE_DATA_CONFIG
                        Write data config files an

In [7]:
!gwnr_write_pycbc_inference_configs --show-available-configs

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
data: ['gw150914-like-gaussian', 'gw150914-like-zeronoise', 'GW150914-v3', 'GW151012-v3', 'GW151226-v2', 'GW170104-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v3', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1']
sampler: ['emcee', 'emcee_pt', 'epsie', 'dynesty', 'ultranest', 'multinest', 'cpnest']
inference: ['bbh_precessing', 'bbh_alignedspin']


In [8]:
!gwnr_write_pycbc_inference_configs --verbose\
  --write-sampler-config emcee_pt\
  --write-inference-config bbh_precessing\
  --write-data-config gw150914-like-zeronoise

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
2021-09-20 12:04:47,882 Writing config file for data settings..
2021-09-20 12:04:47,883 Writing config file for sampler settings..
2021-09-20 12:04:47,883 Writing config file for inference settings..
2021-09-20 12:04:47,883 Done


In [9]:
!ls

bbh_precessing.ini  emcee_pt.ini  gw150914-like-zeronoise.ini


In [10]:
!cat bbh_precessing.ini

[model]
name = gaussian_noise
low-frequency-cutoff = 20.0

[variable_params]
; waveform parameters that will vary in MCMC
delta_tc =
mass1 =
mass2 =
spin1_a =
spin1_azimuthal =
spin1_polar =
spin2_a =
spin2_azimuthal =
spin2_polar =
distance =
coa_phase =
inclination =
polarization =
ra =
dec =

[static_params]
; waveform parameters that will not change in MCMC
approximant = IMRPhenomPv2
f_lower = 20
f_ref = 20
; we'll set the tc by using the trigger time in the data
; section of the config file + delta_tc
trigger_time = ${data|trigger-time}

[prior-delta_tc]
; coalescence time prior
name = uniform
min-delta_tc = -0.1
max-delta_tc = 0.1

[waveform_transforms-tc]
; we need to provide tc to the waveform generator
name = custom
inputs = delta_tc
tc = ${data|trigger-time} + delta_tc

;Mass1 of GW151012 $\in$ [28.7, 38.1]
;Mass1 of GW170608 $\in$ [12.7, 16.5]
;Mass1 of GW170729 $\in$ [60.4, 66.4]
;Mass1 of GW150914 $\in$ [38.7, 40.3]
;Mass1 of GW151226 $\in$ [16.9, 22.5]
;Mass1 of GW170814 

In [11]:
!cat emcee_pt.ini

[sampler]
name = emcee_pt
nwalkers = 500
ntemps = 20
;##### Other possible options
effective-nsamples = 4000
checkpoint-interval = 2000
max-samples-per-chain = 1000

[sampler-burn_in]
burn-in-test = nacl & max_posterior

;
;   Sampling transforms
;
[sampling_params]
; parameters on the left will be sampled in
; parametes on the right
mass1, mass2 : mchirp, q

[sampling_transforms-mchirp+q]
; inputs mass1, mass2
; outputs mchirp, q
name = mass1_mass2_to_mchirp_q


In [12]:
!cat gw150914-like-zeronoise.ini

[data]
instruments = H1 L1
trigger-time = 1126259462.42
analysis-start-time = -6
analysis-end-time = 2
; strain settings
sample-rate = 2048
fake-strain = H1:zeroNoise L1:zeroNoise
; psd settings
psd-model = aLIGOZeroDetHighPower
psd-inverse-length = 0
; even though we're making fake strain, the strain
; module requires a channel to be provided, so we'll
; just make one up
channel-name = H1:STRAIN L1:STRAIN
; Providing an injection file will cause a simulated
; signal to be added to the data
injection-file = injection.hdf
; We'll use a high-pass filter so as not to get numerical errors from the large
; amplitude low frequency noise. Here we use 15 Hz, which is safely below the
; low frequency cutoff of our likelihood integral (20 Hz)
strain-high-pass = 15
; The pad-data argument is for the high-pass filter: 8s are added to the
; beginning/end of the analysis/psd times when the data is loaded. After the
; high pass filter is applied, the additional time is discarded. This pad is
; *in ad

## Write workflow config

In [13]:
# Write CONFIGS
with open("injection.ini", "w") as fout:
    fout.write("""\
[static_params]
tc = 1126259462.420
;mass1 = 37
mass2 = 32
ra = 2.2
dec = -1.25
inclination = 2.5
coa_phase = 1.5
polarization = 1.75
distance = 100
f_ref = 20
f_lower = 18
approximant = SEOBNRv4
taper = start

[variable_params]
mass1 =
eccentricity =
mean_per_ano =

[prior-mass1]
name = uniform
min-mass1 = 10.
max-mass1 = 80.

[prior-eccentricity]
name = uniform
min-eccentricity = 0.
max-eccentricity = 0.2

[prior-mean_per_ano]
name = uniform
min-mean_per_ano = 0.
max-mean_per_ano = 3.1416
""")

In [14]:
!ls

bbh_precessing.ini  emcee_pt.ini  gw150914-like-zeronoise.ini  injection.ini


In [15]:
# Workflow.ini
with open("config.ini", "w") as fout:
    fout.write("""\
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inspinj = ${which:pycbc_create_injections}
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
templates-per-job = 100
log-path = log
banksim-request-memory = 8G
data = gw150914-like-zeronoise.ini
sampler = emcee_pt.ini
inference = bbh_precessing.ini

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Injections
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inspinj]
config-files = injection.ini
ninjections = 10
seed = 10
output-file = injection.hdf
variable-params-section = variable_params
static-params-section = static_params
dist-section prior =
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
output-file = inference.hdf
nprocesses = 10
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Visualize
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[plot]
input-file = inference.hdf
output-file = plots/posteriors.png
plot-scatter =
plot-marginal =
plot-prior = inference.ini data.ini
""")

In [16]:
!cat injection.ini

[static_params]
tc = 1126259462.420
;mass1 = 37
mass2 = 32
ra = 2.2
dec = -1.25
inclination = 2.5
coa_phase = 1.5
polarization = 1.75
distance = 100
f_ref = 20
f_lower = 18
approximant = SEOBNRv4
taper = start

[variable_params]
mass1 =
eccentricity =
mean_per_ano =

[prior-mass1]
name = uniform
min-mass1 = 10.
max-mass1 = 80.

[prior-eccentricity]
name = uniform
min-eccentricity = 0.
max-eccentricity = 0.2

[prior-mean_per_ano]
name = uniform
min-mean_per_ano = 0.
max-mean_per_ano = 3.1416


In [17]:
!cat config.ini

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Executables
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[executables]
inspinj = ${which:pycbc_create_injections}
inference = ${which:pycbc_inference}
plot = ${which:pycbc_inference_plot_posterior}

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Workflow
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[workflow]
accounting-group = ligo.dev.o3.cbc.explore.test
templates-per-job = 100
log-path = log
banksim-request-memory = 8G
data = gw150914-like-zeronoise.ini
sampler = emcee_pt.ini
inference = bbh_precessing.ini

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Injections
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inspinj]
config-files = injection.ini
ninjections = 10
seed = 10
output-file = injection.hdf
variable-params-section = variable_params
static-params-section = static_params
dist-section prior =
force =

;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
;   Inference
;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
[inference]
verbose =
seed = 12
config-files = inference.ini data.ini sampler.ini
ou

## Generate workflow

In [17]:
!gwnr_create_injections_pycbc_inference_workflow -h

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_create_injections_pycbc_inference_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of simulated signals

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --skip-creating-injections
                        Skip calling lalapps_inspinj and assume injections
                      

In [19]:
!gwnr_create_injections_inference_workflow --config-files config.ini\
    --output-dir . --force --verbose

Could not import ligolw in /home/prayush/src/GWNRTools/GWNRTools/Stats/FisherMatrixUtilities.pyc, LIGO XML tables wont be read
2020-03-04 18:50:57,970 Using seed 0
2020-03-04 18:50:57,971 Will setup analyses in .
2020-03-04 18:50:57,971 Running with CPU support: 1 threads
2020-03-04 18:50:58,056 Reading configuration file
2020-03-04 18:50:58,057 Making workspace directories
2020-03-04 18:50:58,073 Creating DAG
2020-03-04 18:50:58,074 Making injection002/gw150914-like-zeronoise/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_inj
2020-03-04 18:50:58,101 Copying config files to injection002/gw150914-like-zeronoise/emcee_pt/bbh_precessing
2020-03-04 18:50:58,102 Copying executables to injection002/gw150914-like-zeronoise/emcee_pt/bbh_precessing/scripts/
2020-03-04 18:50:58,104 Making injection003/gw150914-like-zeronoise/emcee_pt/bbh_precessing in /home/prayush/research/test_pycbc_inj
2020-03-04 18:50:58,131 Copying config files to injection003/gw150914-like-zeronoise/emcee_pt/

In [20]:
!tree

.
├── bbh_precessing.ini
├── config.ini
├── emcee_pt.ini
├── gw150914-like-zeronoise.ini
├── injection000
│   └── gw150914-like-zeronoise
│       └── emcee_pt
│           └── bbh_precessing
│               ├── data.ini
│               ├── inference.ini
│               ├── injection.ini
│               ├── log
│               ├── make_injection
│               ├── make_injection.sub
│               ├── make_plot
│               ├── make_plot.sub
│               ├── plots
│               ├── run_inference
│               ├── run_inference.sub
│               ├── sampler.ini
│               └── scripts
│                   ├── pycbc_create_injections
│                   ├── pycbc_inference
│                   └── pycbc_inference_plot_posterior
├── injection001
│   └── gw150914-like-zeronoise
│       └── emcee_pt
│           └── bbh_precessing
│               ├── data.ini
│               ├── inference.ini
│               ├── injection.ini
│               ├── log
│               ├── make_inj

## Submit & monitor workflow

In [21]:
!condor_submit_dag pycbc_inference_injections.dag >> dag.out

In [24]:
!tail *dagman.out

03/04/20 18:51:03 Number of idle job procs: 4
03/04/20 18:51:03 Reassigning the id of job bef8a4cbe1f89655c581f3272242dc6f from (148.0.0) to (148.0.0)
03/04/20 18:51:03 Event: ULOG_SUBMIT for HTCondor Node bef8a4cbe1f89655c581f3272242dc6f (148.0.0) {03/04/20 18:51:03}
03/04/20 18:51:03 Number of idle job procs: 5
03/04/20 18:51:03 DAG status: 0 (DAG_STATUS_OK)
03/04/20 18:51:03 Of 30 nodes total:
03/04/20 18:51:03  Done     Pre   Queued    Post   Ready   Un-Ready   Failed
03/04/20 18:51:03   ===     ===      ===     ===     ===        ===      ===
03/04/20 18:51:03     0       0        5       0       5         20        0
03/04/20 18:51:03 0 job proc(s) currently held


In [23]:
!ls

bbh_precessing.ini	     injection009
config.ini		     injection.ini
dag.out			     log
emcee_pt.ini		     plots
gw150914-like-zeronoise.ini  pycbc_inference_injections.dag
injection000		     pycbc_inference_injections.dag.condor.sub
injection001		     pycbc_inference_injections.dag.dagman.log
injection002		     pycbc_inference_injections.dag.dagman.out
injection003		     pycbc_inference_injections.dag.lib.err
injection004		     pycbc_inference_injections.dag.lib.out
injection005		     pycbc_inference_injections.dag.lock
injection006		     pycbc_inference_injections.sh
injection007		     scripts
injection008


# Setup and run Bayesian inference on a batch of GW injections using `bilby`

## Setup

In [2]:
import os

run_dir = '/home/prayush/research/test_bilby_inj'
try:
    os.makedirs(run_dir)
except:
    pass
os.chdir(run_dir)

In [3]:
!pwd

/home/prayush/research/test_bilby_inj


In [4]:
!rm -fr *

In [5]:
!ls

## Generate workflow config files

In [20]:
!ls

bbh-alignedspin-prior-aligned_spins_bbh.ini  emcee_pt.ini
bbh_precessing.ini			     gw150914-like-zeronoise.ini


In [21]:
!rm -rf *

In [22]:
!gwnr_write_bilby_configs --help

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_write_bilby_configs [--options]

Get and write configuration files for generating a workflow to perform
Bayesian parameter estimation runs on a custom set of signals with Bilby

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --write-config WRITE_CONFIG
                        Comma-separated name of config-type,config-subtyp

In [23]:
!gwnr_write_bilby_configs --show-available-configs

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
prior: ['default', 'eccentric-nonspin', 'precessing_spins_bns', 'GW170817', 'GW150914', 'precessing_spins_bbh_tides_on', 'aligned_spins_bbh_tides_on', 'precessing_spins_bns_tides_on', 'aligned_spins_bbh', 'precessing_spins_bbh', 'aligned_spins_bns', 'aligned_spins_bns_tides_on']
injection: ['bbh-prior-default', 'bbh-alignedspin-prior-default', 'bbh-eccentric-nonspin-prior-default', 'bbh-prior-eccentric-nonspin', 'bbh-alignedspin-prior-eccentric-nonspin', 'bbh-eccentric-nonspin-prior-eccentric-nonspin', 'bbh-prior-precessing_spins_bns', 'bbh-alignedspin-prior-precessing_spins_bns', 'bbh-

In [24]:
!gwnr_write_bilby_configs --write-config injection,bbh-alignedspin-prior-aligned_spins_bbh

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)


In [25]:
!cat bbh-alignedspin-prior-aligned_spins_bbh.ini


[workflow]
tag = test
accounting-group = ligo.dev.o3.cbc.explore.test
log-path = log
request-memory = 2G
request-cpus = 1

[inference]
duration = 4
sample_rate = 2048
lower_frequency_cutoff = 30
upper_frequency_cutoff = 1024
reference_frequency = 30
phase_marginalization =
time_marginalization =
;distance_marginalization =

[sampler]
name = dynesty
npoints = 2000
maxmcmc = 2000
n_check_point = 1000


[data]
analysis_type = event
source_type = bbh
event_name = GW150914
interferometers = H1,L1


[injection]
noise_type = gaussian
asd-file = H1:PATH,L1:PATH
approximant = IMRPhenomPv2
taper = start
num_injections = 10


[injection-static_params]
geocent_time = 1126259462.420
;mass_1 = 37
mass_2 = 32
chi_1 = 0
;chi_2 = 0
ra = 2.2
dec = -1.25
theta_jn = 2.5
phase = 1.5
psi = 1.75
luminosity_distance = 1000

[injection-variable_params]
mass_1 =
chi_2 =


[injection-prior_mass_1]
name = uniform
min-mass_1 = 10.
max-mass_1 = 80.


[injection-prior_chi_2]
name = uniform
min-chi_2 = -0.2
max-chi_

In [26]:
# Now edit this workflow config file to your desired settings
# All options that are illustrated in this file should be filled-in
# The pipeline wrapper will consciously not use any "reasonable defaults".

## Generate workflow

In [27]:
!gwnr_create_injections_bilby_workflow --help

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
usage: /home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/gwnrtools-2020.10.30-py3.9.egg/EGG-INFO/scripts/gwnrtools_create_injections_bilby_workflow [--options]

Setup workflow to perform Bayesian parameter estimation runs on a custom set
of simulated signals

optional arguments:
  -h, --help            show this help message and exit
  --version             Prints version information.
  --verbose             Print logging messages.
  --config-file CONFIG_FILE
                        Configuration file with details of analyses. See
                        `gwnrtools_

In [28]:
!ls

bbh-alignedspin-prior-aligned_spins_bbh.ini


In [30]:
!gwnr_create_injections_bilby_workflow --config-file bbh-alignedspin-prior-aligned_spins_bbh.ini\
    --output-dir . --force --verbose

/home/prayush/miniconda3/envs/lalsuite-dev/lib/python3.9/site-packages/pandas/core/common.py:208: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = np.asarray(values, dtype=dtype)
2021-09-20 13:03:07,754 Using seed 0
cp: 'bbh-alignedspin-prior-aligned_spins_bbh.ini' and './bbh-alignedspin-prior-aligned_spins_bbh.ini' are the same file
2021-09-20 13:03:07,763 Will setup analyses in .
2021-09-20 13:03:07,764 Creating DAG
2021-09-20 13:03:07,764 --- verifying injection params config
2021-09-20 13:03:07,764 --- injection params config verified
2021-09-20 13:03:07,764 --- sampling 10 injection params
2021-09-20 13:03:09,210 --- injection params sampled
2021-09-20 13:03:09,210 --- reading injection params
2021-09-20 13:03:09,217 --- 10 injection params read
2021-09-20 

In [31]:
!tree

.
├── bbh-alignedspin-prior-aligned_spins_bbh.ini
├── bilby_injections.dag
├── bilby_injections.sh
├── injection000
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection001
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection002
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection003
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection004
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection005
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection006
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection007
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection008
│   ├── log
│   ├── priors.prior
│   ├── run_inference
│   └── run_inference.sub
├── injection009
│ 

## Submit and monitor workflow

In [32]:
!ls

bbh-alignedspin-prior-aligned_spins_bbh.ini  injection002  injection007
bilby_injections.dag			     injection003  injection008
bilby_injections.sh			     injection004  injection009
injection000				     injection005  injection.hdf
injection001				     injection006  injection.ini


In [33]:
!condor_submit_dag bilby_injections.dag >> dag.out

In [34]:
!tail *dagman.out

09/20/21 17:51:28 Number of idle job procs: 9
09/20/21 17:51:28 Reassigning the id of job 7da849fe15e4ede711c49fb84e5e9cf2 from (109.0.0) to (109.0.0)
09/20/21 17:51:28 Event: ULOG_SUBMIT for HTCondor Node 7da849fe15e4ede711c49fb84e5e9cf2 (109.0.0) {09/20/21 17:51:28}
09/20/21 17:51:28 Number of idle job procs: 10
09/20/21 17:51:28 DAG status: 0 (DAG_STATUS_OK)
09/20/21 17:51:28 Of 10 nodes total:
09/20/21 17:51:28  Done     Pre   Queued    Post   Ready   Un-Ready   Failed
09/20/21 17:51:28   ===     ===      ===     ===     ===        ===      ===
09/20/21 17:51:28     0       0       10       0       0          0        0
09/20/21 17:51:28 0 job proc(s) currently held


In [35]:
!ls

bbh-alignedspin-prior-aligned_spins_bbh.ini  injection001
bilby_injections.dag			     injection002
bilby_injections.dag.condor.sub		     injection003
bilby_injections.dag.dagman.log		     injection004
bilby_injections.dag.dagman.out		     injection005
bilby_injections.dag.lib.err		     injection006
bilby_injections.dag.lib.out		     injection007
bilby_injections.dag.lock		     injection008
bilby_injections.dag.nodes.log		     injection009
bilby_injections.sh			     injection.hdf
dag.out					     injection.ini
injection000
